# TEMPLATE JAWABAN TUGAS LANGUAGE MODELING

### Import Library

In [1]:
import nltk
import xlrd
import operator
import random
import math
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Import Dataset (Training dan Testing)

In [16]:
book = xlrd.open_workbook('dataTrainArticle.xlsx')
book2 = xlrd.open_workbook('dataTestArticle.xlsx')
sheet = book.sheet_by_name('Sheet1')
sheet2 = book2.sheet_by_name('Sheet1')
dataTrainDefault = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]
dataTestDefault = [[sheet2.cell_value(r, c) for c in range(sheet2.ncols)] for r in range(sheet2.nrows)]
#split each sentence in each article
dataTrain = []
for article in dataTrainDefault:
    sentence = article[0].split('.')
    for word in sentence:
        dataTrain.append(word)
dataTest = []
for article in dataTestDefault:
    sentence = article[0].split('.')
    for word in sentence:
        dataTest.append(word)
#add dummy-ish data for OOV (word that doesn't exist in data train), the OOV will be converted to _____
#just 4 basic word classes, noun, verbs, adjectives, adverb
#without this p(oov) will be 0
dataTrain.append('the _____ has _____ a new car that will drive arond the city _____') 
#the <noun> has <verb> a new car that will drive around the city <adverb>
dataTrain.append('_____  is one _____ car')
#<noun> has <verb> a new car that will drive around the city <adverb>
print('done')

done


### Bangun Model Unigram

In [17]:
def buildUnigramModel(trainingSentences):
    wordCount = {}
    count = 0
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence)
        count += 2
        wordCount['<s>'] += 1
        wordCount['</s>'] += 1
        for token in tokens:
            tokenLower = token.lower()
            if tokenLower in wordCount:
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
            count += 1
    unigramModel = wordCount
    #count prob
    for word in unigramModel:
        unigramModel[word] = unigramModel[word] / count
    return unigramModel
unigramModel = buildUnigramModel(dataTrain)
print('done')

done


### Bangun Model Bigram

In [18]:
def buildBigramModel(trainingSentences):
    wordPairCount = {}
    wordCount = {}
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    wordPairCount['<s>'] = {}
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence)
        counter = 0
        tokenBefore = ''
        wordCount['<s>'] += 1
        wordCount['</s>'] += 1
        for token in tokens:
            tokenLower = token.lower()
            counter += 1
            if tokenLower not in wordPairCount:
                wordPairCount[tokenLower] = {}
            if counter == 1:
                if tokenLower in wordPairCount['<s>']:
                    wordPairCount['<s>'][tokenLower] += 1
                else:
                    wordPairCount['<s>'][tokenLower] = 1
            elif counter == len(tokens):
                if tokenLower in wordPairCount[tokenBefore]:
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
                if '</s>' in wordPairCount[tokenLower]:
                    wordPairCount[tokenLower]['</s>'] += 1
                else:
                    wordPairCount[tokenLower]['</s>'] = 1
            else:
                if tokenLower in wordPairCount[tokenBefore]:
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
            tokenBefore = tokenLower
            if tokenLower in wordCount:
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
    bigramModel = {}
    
    vee = len(wordCount) #for smoothing, the V
    #smoothing (adding k)
    kay = 0.1 #the k for smoothing
    for wordBefore in wordPairCount.keys():
        for theWord in wordCount.keys():
            if theWord != '<s>':
                if theWord not in wordPairCount[wordBefore]:
                    wordPairCount[wordBefore][theWord] = kay
                else:
                    wordPairCount[wordBefore][theWord] += kay
                #bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    #smoothing (adding vee)
    for theWord in wordCount.keys():
        wordCount[theWord] += (vee * kay)
    #count prob
    for wordBefore in wordPairCount.keys():
        bigramModel[wordBefore] = {}
        for theWord in wordCount.keys():
            if theWord != '<s>':
                bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    '''
    for wordBefore in wordPairCount.keys():
        bigramModel[wordBefore] = {}
        for theWord in wordPairCount[wordBefore].keys():
            if theWord != '<s>':
                bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    '''
    return bigramModel
bigramModel = buildBigramModel(dataTrain)
print('done')

done


### Bangun Fungsi Kata Berikutnya Dengan Probability Tertinggi

In [40]:
#Diberikan sebuah kata, cari kata berikutnya yang memiliki probability tertinggi berdasarkan model bigram
#random from top n to spice things up
def nextBestWord(bigramModel, currentWord, topn):
    top = []
    holder = {}
    currentWordLow = currentWord.lower()
    if currentWordLow not in bigramModel:
        currentWordLow = "_____"
    for i in range(topn):
        maxs = max(bigramModel[currentWordLow].items(), key=operator.itemgetter(1))[0]
        holder[maxs] = bigramModel[currentWordLow][maxs]
        top.append(maxs)
        bigramModel[currentWordLow].pop(maxs)
    for i in range(topn):
        bigramModel[currentWordLow].update( {top[i] : holder[top[i]]} )
    maks = sum(holder.values())
    pick = random.uniform(0, maks)
    current = 0
    for key, value in holder.items():
        current += value
        if current > pick:
            return key
    #return nextWord
#Lakukan percobaan dengan memberikan sebuah kata, lalu print kata berikutnya
print(nextBestWord(bigramModel,"lexus",3))

is


### Hitung Perplexity Unigram dan Bigram terhadap Data Testing

In [38]:
def countPerplexity(unigramModel, bigramModel, testSentences):
    perplexity = {}
    #unigram
    count1 = 0
    total1 = 0
    for sentence in testSentences:
        pos = 0
        tokens = nltk.word_tokenize(sentence)
        for word in tokens:
            pos += 1
            count1 += 1
            wordLow = word.lower()
            if word not in unigramModel:
                wordLow = "_____"
            if pos > 1:
                total1 += math.log2(unigramModel[wordLow])
            elif pos == len(tokens):
                total1 += math.log2(unigramModel[wordLow])
                total1 += math.log2(unigramModel['</s>'])
                count1 += 1
            elif pos == 1:
                count1 += 1
                total1 += math.log2(unigramModel['<s>'])
                total1 += math.log2(unigramModel[wordLow])
    total1 = total1 / count1
    total1 = total1 * -1
    perplexity1 = 2 ** total1
    perplexity.update({'unigram' : perplexity1})
    #bigram
    count2 = 0
    total2 = 0
    for sentence in testSentences:
        pos = 0
        tokens = nltk.word_tokenize(sentence)
        for word in tokens:
            count2 += 1
            pos += 1
            wordLow = word.lower()
            if word not in bigramModel:
                wordLow = "_____"
            if pos > 1:
                total2 += math.log2(bigramModel[wordBefore][wordLow])
            elif pos == len(tokens):
                total2 += math.log2(bigramModel[wordBefore][wordLow])
                total2 += math.log2(bigramModel[wordLow]['</s>'])
                count2 += 1
            elif pos == 1:
                total2 += math.log2(bigramModel['<s>'][wordLow])
            wordBefore = wordLow
    total2 = total2 / count2
    total2 = total2 * -1
    perplexity2 = 2 ** total2
    perplexity.update({'bigram' : perplexity2})
    return perplexity
print(countPerplexity(unigramModel, bigramModel, dataTest))

{'unigram': 773.3640489611981, 'bigram': 1196.055093998808}


### Bangun Fungsi Generate Kalimat

In [83]:
def generateSentence(model, length):
    sentence = ''
    wordBefore = '<s>'
    for i in range(length):
        nextWord = nextBestWord(model,wordBefore,2)
        if nextWord == '<s>':
            sentence += ' '
        elif nextWord == "</s>":
            sentence += '.'
        else:
            if nextWord != "'" and nextWord != '’' and wordBefore != '’' and nextWord != '"' and nextWord != ',' and nextWord != '(' and nextWord != ')':
                sentence += ' '
            sentence += nextWord
        wordBefore = nextWord
    return sentence
generateSentence(bigramModel,100)

' it was built on the car that’s a little more than a little more powerful than the new car will be able to be built in the new car that the new car that’s a little more powerful than a little more than the car will be able baton studied its new car will make it’s not so far as a little too late 1920s and the new corsa is a new corsa gsi. car that the car will be able to be built in the car will be built in the car'